daisyRec version 1.1.2

In [1]:
import daisy.utils

In [2]:
from daisy.utils.metrics import *

In [3]:
import pandas as pd

In [4]:
df_pred = pd.read_csv('./pred.csv')

df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [5]:
df_pred_new.head()

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0


In [6]:
df_test.head()

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5.0
1,1256,778,4.5,2007-10-31 12:19:51,4.5
2,1256,8376,5.0,2007-10-31 12:30:59,5.0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5


In [7]:
recommendations = {}

for user, item_id in zip(df_pred_new.user_id, df_pred_new.relevance):
    if user in recommendations:
        recommendations[user] += [item_id]
    else:
        recommendations[user] = [item_id] 

In [8]:
gt_recommendations = {}

for user, item_id in zip(df_test.user_id, df_test.item_id):
    if user in gt_recommendations:
        gt_recommendations[user] += [item_id]
    else:
        gt_recommendations[user] = [item_id] 

MAP из коробки падает:

In [9]:

map_at_k(recommendations)

IndexError: invalid index to scalar variable.

In [10]:
from daisy.utils.metrics import ap

если пофиксим очевидную ошибку в MAP, то получаем:

In [11]:
def map_at_k(rs):
    """
    Mean Average Precision calculation method
    Parameters
    ----------
    rs : Dict, {user : rank items} for test set
    Returns
    -------
    m_a_p : float, MAP value
    """
    m_a_p = np.mean([ap(r) for r in rs.items()])
    return m_a_p

In [12]:
map_at_k(recommendations)

1.0

Если внести еще изменение, получим то же, что и в Replay:

In [13]:
def ap(r, user):
    """
    Average precision calculation method
    Parameters
    ----------
    r : List, Relevance scores (list or numpy) in rank order (first element is the first item)
    Returns
    -------
    a_p : float, Average precision value
    """
    r = np.asarray(r) != 0
    out = [r[k] * precision_at_k(r, k + 1) for k in range(r.size)]
#     if not out:
#         return 0.

#     max_good = len(tmp_test[user])
#     max_good = len(r)
    max_good = min(len(gt_recommendations[user]), len(r))
    a_p = np.sum(out) / max_good

    return a_p


def map_at_k(rs):
    """
    Mean Average Precision calculation method
    Parameters
    ----------
    rs : Dict, {user : rank items} for test set
    Returns
    -------
    m_a_p : float, MAP value
    """
    a = [ap(r, user) for user, r in rs.items()]
    m_a_p = np.mean(a)
    return m_a_p

In [14]:
map_at_k(recommendations)

0.03927176078866248

In [15]:
hr_at_k(recommendations, gt_recommendations)

0.47457006952067327

In [16]:
np.mean([precision_at_k(x, 20) for x in list(recommendations.values())])

0.057519209659714604

df_test_only - датасет, где лежат ground truth, которые не встретились в топе предиктов. 

In [17]:
df_test_only = pd.merge(df_test, df_pred_new, on=('user_id', 'item_id'), how='left')
df_test_only = df_test_only[df_test_only.score.isna()]

In [18]:
np.mean([
    ndcg_at_k(val + [1] *  df_test_only[df_test_only.user_id == user].shape[0], 20) 
    for user, val in recommendations.items()
])

0.09326437896736814

In [19]:
recall_at_k(recommendations, gt_recommendations, 20)

0.09632131280310641

In [20]:
mrr_at_k(recommendations, 20)

0.2748816881385204

In [21]:
auc_at_k(recommendations)

ZeroDivisionError: float division by zero

AUC падает в случае, когда нет либо негативов, либо позитивов: в знаменателе (pos_num * neg_num)